In [16]:
import pandas as pd
import xgboost as xgb

## Import Data

In [8]:
boston = pd.read_csv("boston_data.csv")
boston.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,0.15876,0.0,10.81,0.0,0.413,5.961,17.5,5.2873,4.0,305.0,19.2,376.94,9.88,21.7
1,0.10328,25.0,5.13,0.0,0.453,5.927,47.2,6.9320,8.0,284.0,19.7,396.90,9.22,19.6
2,0.34940,0.0,9.90,0.0,0.544,5.972,76.7,3.1025,4.0,304.0,18.4,396.24,9.97,20.3
3,2.73397,0.0,19.58,0.0,0.871,5.597,94.9,1.5257,5.0,403.0,14.7,351.85,21.45,15.4
4,0.04337,21.0,5.64,0.0,0.439,6.115,63.0,6.8147,4.0,243.0,16.8,393.97,9.43,20.5


In [6]:
boston.shape

(404, 14)

In [7]:
boston.dtypes

crim       float64
zn         float64
indus      float64
chas       float64
nox        float64
rm         float64
age        float64
dis        float64
rad        float64
tax        float64
ptratio    float64
black      float64
lstat      float64
medv       float64
dtype: object

In [9]:
X = boston.iloc[:, :13]
y = boston['medv']

In [13]:
print(X.shape)
print(y.shape)

(404, 13)
(404,)


In [14]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat
354,1.15172,0.0,8.14,0.0,0.5380,5.701,95.0,3.7872,4.0,307.0,21.0,358.77,18.35
234,4.55587,0.0,18.10,0.0,0.7180,3.561,87.9,1.6132,24.0,666.0,20.2,354.70,7.12
3,2.73397,0.0,19.58,0.0,0.8710,5.597,94.9,1.5257,5.0,403.0,14.7,351.85,21.45
18,1.65660,0.0,19.58,0.0,0.8710,6.122,97.3,1.6180,5.0,403.0,14.7,372.80,14.10
131,0.08308,0.0,2.46,0.0,0.4880,5.604,89.8,2.9879,3.0,193.0,17.8,391.00,13.98
118,0.16760,0.0,7.38,0.0,0.4930,6.426,52.3,4.5404,5.0,287.0,19.6,396.90,7.20
327,0.35114,0.0,7.38,0.0,0.4930,6.041,49.9,4.7211,5.0,287.0,19.6,396.90,7.70
63,0.04932,33.0,2.18,0.0,0.4720,6.849,70.3,3.1827,7.0,222.0,18.4,396.90,7.53
317,0.44178,0.0,6.20,0.0,0.5040,6.552,21.4,3.3751,8.0,307.0,17.4,380.34,3.76
211,0.04981,21.0,5.64,0.0,0.4390,5.998,21.4,6.8147,4.0,243.0,16.8,396.90,8.43


In [17]:
xgbr = xgb.XGBRegressor()

In [18]:
xgbr.fit(X_train, y_train)

[15:59:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/home/ccc/anaconda3/envs/mlnote/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [19]:
print('The accuracy of eXtreme Gradient Boosting Regressor on testing set: ', xgbr.score(X_test, y_test))

The accuracy of eXtreme Gradient Boosting Regressor on testing set:  0.7140740031267654
